In [40]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

In [41]:
food=pd.read_csv('./1662574418893344.csv')
ratings=pd.read_csv('./ratings.csv')
food.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [42]:
dataset=ratings.pivot_table(index='Food_ID',columns='User_ID' ,values='Rating')
dataset.fillna(0,inplace=True)
dataset.head()

User_ID,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
Food_ID,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0


In [53]:
def food_rec(Food_Name):
    n=5
    Food_List=food[food['Name'].str.contains(Food_Name)]#The .contains() method is specifically used to check whether a given substring (Food_Name) exists within each string element of the column (food['Name']).
    if len(Food_List):
        Foodid=Food_List.iloc[0]['Food_ID']
        Foodid=dataset[dataset['Food_ID']==Foodid].index[0]
        distances, indices=model.kneighbors(csr_dataset[Foodid],n_neighbors=n+1)
        Food_indices=sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x:x[1])
        recom=[]
        for val in Food_indices[1:]:
            Foodid=dataset.iloc[val[0]]['Food_ID']
            i=food[food['Food_ID']==Foodid].index[0]
            recom.append({'Name':food.iloc[i]['Name'],'Distance':val[1]})
        df=pd.DataFrame(recom)
        return df['Name']
    else:
        return "NO SIMILAR FOOD"
csr_dataset=csr_matrix(dataset.values)
if 'level_0' in dataset.columns:
    dataset.drop(columns=['level_0'], inplace=True)
dataset.reset_index(inplace=True)
model=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model.fit(csr_dataset)
food_rec('soup')

0    lemon honey glazed sous vide corn on the cob
1                            hawaiin papaya salad
2            chicken parmigiana with tomato sauce
3        roasted spring chicken with root veggies
4                            dahi lasooni chicken
Name: Name, dtype: object